In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
from itertools import chain, product
from pathlib import Path
from typing import Union, Tuple, Optional
import pandas as pd
import geopandas as gpd
import numpy as np
from osgeo import gdal
from shapely.geometry import Point
from pysheds.grid import Grid


In [3]:
def create_mosaic(file_path: Union[Path, str], file_name: str, tiles: list) -> str:
    """Generate vrt mosaic for GDAL procedure from each .tiff for extent.

    Args:
    ----
        file_path (Union[Path, str]): Path for results
        file_name (str): Name for file (without extension)
        tiles (list): List of .tiff files from which we'll crop result

    Returns:
    -------
        str: Path for created mosaic
    """
    file_target = f"{file_path}/{file_name}.vrt"
    mosaic = gdal.BuildVRT(destName=file_target, srcDSOrSrcDSTab=tiles)
    mosaic.FlushCache()

    return file_target


def vrt_to_geotiff(vrt_path: str, geotiff_path: str):
    src_ds = gdal.Open(vrt_path, 0)  # open the VRT in read-only mode
    gdal.Translate(
        geotiff_path,
        src_ds,
        format="GTiff",
        creationOptions=["COMPRESS:DEFLATE", "TILED:YES"],
        callback=gdal.TermProgress_nocb,
    )
    # Properly close the datasets to flush to disk
    src_ds = None
    return geotiff_path


def gdal_extent_clipper(
    initial_tiff: str, extent: tuple, tmp_tiff: str, final_tiff: str, crs_epsg: int
) -> None:
    """Clip and reproject .tiff file for desired extent and epsg.

    Args:
    ----
        initial_tiff (str): Ininitial .tiff file which need to be trimmed
        extent (tuple): Min, max coordinates for area of interest
        tmp_tiff (str): Name for file in temporary folder (trimmed, not projected)
        final_tiff (str): Name for projected and trimmed file
        crs_epsg (int): EPSG code

    Returns:
    -------
        None
    """
    # clip big tiff for extent
    merged_tiff = gdal.Translate(destName=tmp_tiff, srcDS=initial_tiff, projWin=extent)
    merged_tiff.FlushCache()
    # reproject for desired CRS
    merged_tiff_proj = gdal.Warp(
        destNameOrDestDS=final_tiff,
        format="GTiff",
        dstNodata=None,
        srcDSOrSrcDSTab=tmp_tiff,
        dstSRS=f"EPSG:{crs_epsg}",
    )
    merged_tiff_proj.FlushCache()

    return None


In [36]:
def gauge_buffer_creator(gauge_geometry: Point):
    """Create squared buffer for extent of flood modelling.

    Args:
    ----
        gauge_geometry (Point): Shapely Point object from geometry column
        ws_gdf (gpd.GeoSeries): Watershed-object based on gauge_id GeoDataFrame
        tiff_epsg (int): Metric EPSG code

    Returns:
    -------
        * Trimmed buffer around point of interest
        * extent coordinates of full-sized buffer
        * number of cells which will be a limit value for innudation mapping
        * watershed area in sq. km

        Tuple[gpd.GeoDataFrame, Tuple[float, float, float, float], float, float]

    """
    AREA_SIZE = 0.5
    # create buffer for point
    buffer = gauge_geometry.buffer(AREA_SIZE, cap_style="square")
    # create buffer for river intersection search
    buffer_isc = gauge_geometry.buffer(AREA_SIZE - 0.015, cap_style="square")
    buffer_gdf = pd.DataFrame(
        columns=["geometry"],
    )
    #    index=[0])
    buffer_gdf = gpd.GeoDataFrame(buffer_gdf, geometry="geometry", crs=4326)  # type: ignore
    buffer_gdf.loc[0, "geometry"] = buffer_isc
    # get coords from buffer
    upper_left_x, lower_right_y, lower_right_x, upper_left_y = buffer.bounds

    wgs_window = (upper_left_x, upper_left_y, lower_right_x, lower_right_y)

    return (buffer_gdf, wgs_window)


def fill_dem(dem_tiff: str, elv_fill: str) -> None:
    """Fill raster elevation pits and depressions.

    Args:
    ----
        dem_tiff (str): Path to initial dem file (.tiff)
        elv_fill (str): Path to filled dem file (.tiff)

    Returns:
    -------
        None

    """
    # Read raw DEM
    grid = Grid.from_raster(dem_tiff)
    dem = grid.read_raster(dem_tiff)
    # Fill pits
    pit_filled_dem = grid.fill_pits(dem)
    # Fill depressions
    flooded_dem = grid.fill_depressions(pit_filled_dem)
    # Resolve flats
    inflated_dem = grid.resolve_flats(flooded_dem)
    inflated_dem[inflated_dem < 0] = np.nan  # type: ignore
    grid.to_raster(file_name=elv_fill, data=inflated_dem)

    return None


In [5]:
tan_dem_files = list(Path("/mnt/f/HydroResearchData/DEM/TanDEM/").glob("*.tif"))

gauges = gpd.read_file("data/geometry/qh_gauges.gpkg")
gauges = gauges.set_index("gauge_id")
gauges

,name_ru,name_en,geometry
gauge_id,,,
19313,р.Худолаз - пос.Чернышевский,r.Hudolaz - pos.Chernyshevskij,POINT (58.82645 52.63939)
75236,р.Касть - д.Рылово,r.Kast' - d.Rylovo,POINT (40.45643 58.01418)
72559,р.Систа - д.Среднее Райково,r.Sista - d.Srednee Rajkovo,POINT (28.8 59.73)
75222,р.Печегда - пос.Чебаково,r.Pechegda - pos.Chebakovo,POINT (39.56053 57.75352)
71044,р.Ура - с.Ура-Губа,r.Ura - s.Ura-Guba,POINT (32.77889 69.26977)
...,...,...,...
77362,р.Большой Караман - пгт Советское,r.Bol'shoj Karaman - pgt Sovetskoe,POINT (46.75 51.45)
3577,р.Чечуй - д.Пущино,r.Chechuj - d.Puschino,POINT (109.33 58.2)
78092,р.Лесной Воронеж - сл.Заворонежская,r.Lesnoj Voronezh - sl.Zavoronezhskaja,POINT (40.53048 52.88205)


In [6]:
gauge_id = "19313"
gauge = gauges.loc[[gauge_id], :]

lon, lat = gauge.loc[gauge_id, "geometry"].xy
lon, lat = lon[0], lat[0]


def round_up(x, round_val: float = 5):
    return int(np.ceil(x / round_val)) * round_val


def round_down(x, round_val: float = 5):
    return int(np.floor(x / round_val)) * round_val


max_lon, min_lon = round_up(lon, round_val=1), round_down(lon, round_val=1)
max_lat, min_lat = round_up(lat, round_val=1), round_down(lat, round_val=1)

gauge_extent = (min_lon, min_lat, max_lon, max_lat)

tile_boundaries = [
    f"N{lat}E0{lon}" if len(str(lon)) < 3 else f"N{lat}E{lon}"
    for lat, lon in product((min_lat, max_lat), (min_lon, max_lon))
]
tiles = list(
    chain.from_iterable(
        [[i for i in tan_dem_files if i.match(f"*{boundary}*")] for boundary in tile_boundaries]
    )
)
elv_vrt_path = create_mosaic(file_path="data/rasters/", file_name=f"{gauge_id}_tan_elv", tiles=tiles)



/home/dmbrmv/miniconda3/envs/geo/lib/python3.10/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [16]:
def gauge_to_utm(
    gauge_series: Point, return_gdf: bool = False
) -> Optional[Union[Tuple[gpd.GeoSeries, int], Point]]:
    """Convert geometry projected file to it metric projection.

    Args:
    ----
        gauge_series (gpd.GeoSeries):
            GeoDataFrame file with geometry in WGS-84

        return_gdf (bool, optional):
            If require just coordinates we return projected Point object.
            Otherwise GeoDataFrame, and EPSG code for further trasnformations.
            Defaults to False.

    Returns:
    -------
        Optional[ Union[Tuple[gpd.GeoSeries, int], Point]]

    """
    gdf_file = gpd.GeoSeries(gauge_series, crs=4326)
    gdf_file = gpd.GeoDataFrame(gdf_file, columns=["geometry"])
    tiff_epsg = gdf_file.estimate_utm_crs().to_epsg()
    gdf_file["x"] = gdf_file["geometry"].x
    gdf_file["y"] = gdf_file["geometry"].y
    gdf_file_crs = gdf_file.to_crs(tiff_epsg)
    if return_gdf:
        return (gdf_file, tiff_epsg)
    return gdf_file_crs["geometry"].values[0]


In [37]:
gauge_projected, gauge_utm = gauge_to_utm(gauge_series=gauge.loc[gauge_id, "geometry"], return_gdf=True)
gauge_buffer, gauge_extent = gauge_buffer_creator(gauge_geometry=gauge.loc[gauge_id, "geometry"])
gdal_extent_clipper(
    initial_tiff=elv_vrt_path,
    extent=gauge_extent,
    tmp_tiff=f"data/rasters/{gauge_id}_elv.tiff",
    final_tiff=f"data/rasters/{gauge_id}_elv.tiff",
    crs_epsg=4326,
)
fill_dem(
    dem_tiff=f"data/rasters/{gauge_id}_elv.tiff",
    elv_fill=f"data/rasters/{gauge_id}_fill.tiff",
)


In [56]:
grid = Grid.from_raster(f"data/rasters/{gauge_id}_fill.tiff")
dem = grid.read_raster(f"data/rasters/{gauge_id}_fill.tiff")
# dem.nodata = np.asarray(dem.nodata, dtype=np.float32)
fdir = grid.flowdir(dem)
grid.to_raster(file_name=f"data/rasters/{gauge_id}_dir.tiff", data=fdir)
# Specify directional mapping
dirmap = (64, 128, 1, 2, 4, 8, 16, 32)

# Compute accumulation
acc = grid.accumulation(fdir, dirmap=dirmap)
grid.to_raster(file_name=f"data/rasters/{gauge_id}_acc.tiff", data=acc)


TypeError: `nodata` value not representable in dtype of array.

In [58]:
grid = Grid.from_raster(f"data/rasters/{gauge_id}_fill.tiff")
dem = grid.read_raster(f"data/rasters/{gauge_id}_fill.tiff")
# Fill pits
pit_filled_dem = grid.fill_pits(dem)
# Fill depressions
flooded_dem = grid.fill_depressions(pit_filled_dem)
# Resolve flats
inflated_dem = grid.resolve_flats(flooded_dem)
inflated_dem[inflated_dem < 0] = np.nan  # type: ignore


TypeError: `nodata` value not representable in dtype of array.

In [64]:
fdir = grid.flowdir(inflated_dem, nodata_out=np.uint8(0))
grid.to_raster(file_name=f"data/rasters/{gauge_id}_dir.tiff", data=fdir)
dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
acc = grid.accumulation(fdir, dirmap=dirmap, nodata_out=np.float64(0))
grid.to_raster(file_name=f"data/rasters/{gauge_id}_acc.tiff", data=acc)